In [1]:
import pandas as pd
import numpy as np

In [2]:
import scipy as sp
from scipy.stats import bartlett
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [3]:
import plotly as px
import plotly.graph_objects as go

# 미세먼지농도별 레벨 구분(상계34동)

In [4]:
dust_data = pd.read_csv("C:/Users/jaehun/Desktop/finedust/Bigcon_new_data/환경기상데이터/상계34동.csv")

In [5]:
dust_data.shape

(365, 7)

In [6]:
dust_data

,Date,flag,pm10,noise,temp,humi,pm25
0,2018-04-01,1.0,71.191667,35.585417,2.799236,44.299306,37.002083
1,2018-04-02,1.0,67.463194,35.783333,5.635833,46.036111,28.035417
2,2018-04-03,1.0,70.217361,35.718750,5.156736,50.127083,23.297917
3,2018-04-04,1.0,19.018297,43.173821,6.753483,54.221675,12.158339
4,2018-04-05,1.0,12.666667,36.423480,-5.677079,63.508036,5.267645
...,...,...,...,...,...,...,...
360,2019-03-27,1.0,81.377083,51.019444,12.520556,59.389583,53.036111
361,2019-03-28,1.0,75.091667,50.842361,11.468889,43.751389,46.912500
362,2019-03-29,1.0,37.279167,50.555556,11.021389,25.837500,19.450694
363,2019-03-30,1.0,38.315972,50.928472,6.029931,69.090278,28.082639


In [7]:
dust_data_very_bad = dust_data[dust_data.pm25 >= 76]
dust_data_very_bad = dust_data_very_bad.reset_index(drop=True)

dust_data_bad = dust_data[(dust_data.pm25 < 76) & (dust_data.pm25 >=36)]
dust_data_bad = dust_data_bad.reset_index(drop=True)

dust_data_normal = dust_data[dust_data.pm25 < 36]
dust_data_normal = dust_data_normal.reset_index(drop=True)

In [8]:
print(dust_data_very_bad.shape)
print(dust_data_bad.shape)
print(dust_data_normal.shape)

(21, 7)
(114, 7)
(230, 7)


In [9]:
dust_data_very_bad['level'] = 'very bad'
dust_data_bad['level'] = 'bad'
dust_data_normal['level'] = 'normal'

In [10]:
new_dust_data = pd.concat([dust_data_very_bad,dust_data_bad,dust_data_normal],ignore_index=True)

In [11]:
new_dust_data = new_dust_data.sort_values('Date')
new_dust_data = new_dust_data.reset_index(drop = True)
new_dust_data

,Date,flag,pm10,noise,temp,humi,pm25,level
0,2018-04-01,1.0,71.191667,35.585417,2.799236,44.299306,37.002083,bad
1,2018-04-02,1.0,67.463194,35.783333,5.635833,46.036111,28.035417,normal
2,2018-04-03,1.0,70.217361,35.718750,5.156736,50.127083,23.297917,normal
3,2018-04-04,1.0,19.018297,43.173821,6.753483,54.221675,12.158339,normal
4,2018-04-05,1.0,12.666667,36.423480,-5.677079,63.508036,5.267645,normal
...,...,...,...,...,...,...,...,...
360,2019-03-27,1.0,81.377083,51.019444,12.520556,59.389583,53.036111,bad
361,2019-03-28,1.0,75.091667,50.842361,11.468889,43.751389,46.912500,bad
362,2019-03-29,1.0,37.279167,50.555556,11.021389,25.837500,19.450694,normal
363,2019-03-30,1.0,38.315972,50.928472,6.029931,69.090278,28.082639,normal


In [12]:
Date_level = new_dust_data[['Date','level']]
Date_level.head()

,Date,level
0,2018-04-01,bad
1,2018-04-02,normal
2,2018-04-03,normal
3,2018-04-04,normal
4,2018-04-05,normal


# 유동인구데이터에 Level Colum 추가

In [13]:
import glob 

In [14]:
population_path = 'C:/Users/고유경/Desktop/고려대/KU-BIG/빅콘테스트/유동인구데이터_수정/'

time_list = glob.glob(population_path + '시간대유동/' + '/*.csv')
age_list = glob.glob(population_path + '성연령유동/' + '/*.csv')

In [19]:
# Data 잘 불러지는지 확인

d2 = pd.read_csv(age_list[0])
print(d2.shape)
d2.head()

(1080, 34)


,STD_YM,STD_YMD,HDONG_CD,HDONG_NM,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,...,WMAN_FLOW_POP_CNT_2529,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U
0,201804,20180401,1111051500,청운효자동,0.05,78.93,403.26,642.66,984.24,1792.30,...,1437.80,1356.08,1598.14,1495.53,1768.21,1611.47,1433.06,1212.56,549.45,811.56
1,201804,20180401,1111053000,사직동,1.25,262.54,1207.22,2177.88,4076.89,6887.72,...,6892.47,5743.10,5817.91,5429.89,6104.03,4998.93,4826.11,4207.45,2160.84,2972.08
2,201804,20180401,1111054000,삼청동,0.00,78.07,255.24,331.09,654.32,1264.86,...,1400.28,1183.11,1234.88,1157.69,1186.72,985.11,844.26,744.60,359.03,421.64
3,201804,20180401,1111055000,부암동,0.00,228.50,753.31,981.54,1615.77,2732.00,...,2153.73,2431.23,3028.24,2944.28,3532.23,3075.55,2785.22,2223.72,1118.48,1498.65
4,201804,20180401,1111056000,평창동,0.00,354.38,1500.63,1963.08,2593.07,4927.24,...,3775.91,4598.39,5661.53,5526.94,7075.02,6899.55,6886.22,5336.01,2629.75,3820.41


In [20]:
people_age = pd.read_csv(age_list[0])

In [21]:
for i in age_list[1:]:
    
    Data = pd.read_csv(i)
    
    people_age = pd.concat([people_age,Data], ignore_index = True)

people_age.shape

(13140, 34)

In [22]:
people_age.columns

Index(['STD_YM', 'STD_YMD', 'HDONG_CD', 'HDONG_NM', 'MAN_FLOW_POP_CNT_0004',
       'MAN_FLOW_POP_CNT_0509', 'MAN_FLOW_POP_CNT_1014',
       'MAN_FLOW_POP_CNT_1519', 'MAN_FLOW_POP_CNT_2024',
       'MAN_FLOW_POP_CNT_2529', 'MAN_FLOW_POP_CNT_3034',
       'MAN_FLOW_POP_CNT_3539', 'MAN_FLOW_POP_CNT_4044',
       'MAN_FLOW_POP_CNT_4549', 'MAN_FLOW_POP_CNT_5054',
       'MAN_FLOW_POP_CNT_5559', 'MAN_FLOW_POP_CNT_6064',
       'MAN_FLOW_POP_CNT_6569', 'MAN_FLOW_POP_CNT_70U',
       'WMAN_FLOW_POP_CNT_0004', 'WMAN_FLOW_POP_CNT_0509',
       'WMAN_FLOW_POP_CNT_1014', 'WMAN_FLOW_POP_CNT_1519',
       'WMAN_FLOW_POP_CNT_2024', 'WMAN_FLOW_POP_CNT_2529',
       'WMAN_FLOW_POP_CNT_3034', 'WMAN_FLOW_POP_CNT_3539',
       'WMAN_FLOW_POP_CNT_4044', 'WMAN_FLOW_POP_CNT_4549',
       'WMAN_FLOW_POP_CNT_5054', 'WMAN_FLOW_POP_CNT_5559',
       'WMAN_FLOW_POP_CNT_6064', 'WMAN_FLOW_POP_CNT_6569',
       'WMAN_FLOW_POP_CNT_70U'],
      dtype='object')

In [23]:
people_age.HDONG_NM.unique()

array(['청운효자동', '사직동', '삼청동', '부암동', '평창동', '무악동', '교남동', '가회동',
       '종로1.2.3.4가동', '종로5.6가동', '이화동', '혜화동', '창신1동', '창신2동', '창신3동',
       '숭인1동', '숭인2동', '월계1동', '월계2동', '월계3동', '공릉1동', '공릉2동', '하계1동',
       '하계2동', '중계본동', '중계1동', '중계4동', '중계2.3동', '상계1동', '상계2동', '상계3.4동',
       '상계5동', '상계6.7동', '상계8동', '상계9동', '상계10동'], dtype=object)

In [28]:
people_age['HDONG_NM'] = people_age['HDONG_NM'].apply(lambda x: re.sub("3.4","34",x))

In [25]:
import re

In [29]:
people_age = people_age.groupby('HDONG_NM').get_group('상계34동')
print(people_age.shape)
people_age.head()

(365, 34)


,STD_YM,STD_YMD,HDONG_CD,HDONG_NM,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,...,WMAN_FLOW_POP_CNT_2529,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U
30,201804,20180401,1135066500,상계34동,0.00,153.25,713.84,1252.03,1335.48,1974.79,...,1510.19,1637.68,2266.34,2205.38,3273.12,3088.10,3618.61,2513.22,1474.12,1762.03
66,201804,20180402,1135066500,상계34동,0.00,148.05,452.80,1301.48,1397.54,1800.54,...,1605.64,1698.75,2180.52,2421.76,3264.91,3016.01,3140.79,2345.18,1515.82,1871.65
102,201804,20180403,1135066500,상계34동,0.00,112.74,418.45,1292.73,1273.00,1879.52,...,1544.06,1664.50,2178.74,2502.02,3333.71,3242.28,3254.22,2525.41,1585.93,1763.33
138,201804,20180404,1135066500,상계34동,2.65,132.11,510.20,1384.89,1222.98,1837.64,...,1510.24,1557.18,2271.59,2485.72,3341.92,2962.14,3234.81,2403.02,1578.96,1628.89
174,201804,20180405,1135066500,상계34동,0.00,97.59,428.40,1255.86,1197.84,1921.00,...,1496.56,1590.45,2134.92,2253.33,3183.49,2716.88,2857.37,2168.00,1360.75,1598.15


In [30]:
people_age = people_age.drop(['STD_YM','HDONG_CD'],axis = 1)
people_age.head()

,STD_YMD,HDONG_NM,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,MAN_FLOW_POP_CNT_3034,MAN_FLOW_POP_CNT_3539,...,WMAN_FLOW_POP_CNT_2529,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U
30,20180401,상계34동,0.00,153.25,713.84,1252.03,1335.48,1974.79,1920.19,2772.93,...,1510.19,1637.68,2266.34,2205.38,3273.12,3088.10,3618.61,2513.22,1474.12,1762.03
66,20180402,상계34동,0.00,148.05,452.80,1301.48,1397.54,1800.54,2068.72,3382.82,...,1605.64,1698.75,2180.52,2421.76,3264.91,3016.01,3140.79,2345.18,1515.82,1871.65
102,20180403,상계34동,0.00,112.74,418.45,1292.73,1273.00,1879.52,2023.47,3159.23,...,1544.06,1664.50,2178.74,2502.02,3333.71,3242.28,3254.22,2525.41,1585.93,1763.33
138,20180404,상계34동,2.65,132.11,510.20,1384.89,1222.98,1837.64,2040.81,3205.01,...,1510.24,1557.18,2271.59,2485.72,3341.92,2962.14,3234.81,2403.02,1578.96,1628.89
174,20180405,상계34동,0.00,97.59,428.40,1255.86,1197.84,1921.00,1910.03,3005.59,...,1496.56,1590.45,2134.92,2253.33,3183.49,2716.88,2857.37,2168.00,1360.75,1598.15


In [31]:
people_age['STD_YMD'] = people_age['STD_YMD'].apply(str)

In [32]:
Date_level.head()

,Date,level
0,2018-04-01,bad
1,2018-04-02,normal
2,2018-04-03,normal
3,2018-04-04,normal
4,2018-04-05,normal


In [33]:
Date_level = Date_level.rename(columns= {'Date' : 'STD_YMD'})
Date_level.head()

,STD_YMD,level
0,2018-04-01,bad
1,2018-04-02,normal
2,2018-04-03,normal
3,2018-04-04,normal
4,2018-04-05,normal


In [34]:
print(people_age.shape)
print(Date_level.shape)

(365, 32)
(365, 2)


In [36]:
age_level = pd.merge(people_age,Date_level)
print(age_level.shape)
age_level.head()

(0, 33)


,STD_YMD,HDONG_NM,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,MAN_FLOW_POP_CNT_3034,MAN_FLOW_POP_CNT_3539,...,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U,level


In [38]:
Date_level["STD_YMD"] = Date_level["STD_YMD"].apply(lambda x: re.sub("-","",x))
Date_level.head()

,STD_YMD,level
0,20180401,bad
1,20180402,normal
2,20180403,normal
3,20180404,normal
4,20180405,normal


In [39]:
age_level = pd.merge(people_age,Date_level)
print(age_level.shape)
age_level.head()

(365, 33)


,STD_YMD,HDONG_NM,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,MAN_FLOW_POP_CNT_3034,MAN_FLOW_POP_CNT_3539,...,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U,level
0,20180401,상계34동,0.00,153.25,713.84,1252.03,1335.48,1974.79,1920.19,2772.93,...,1637.68,2266.34,2205.38,3273.12,3088.10,3618.61,2513.22,1474.12,1762.03,bad
1,20180402,상계34동,0.00,148.05,452.80,1301.48,1397.54,1800.54,2068.72,3382.82,...,1698.75,2180.52,2421.76,3264.91,3016.01,3140.79,2345.18,1515.82,1871.65,normal
2,20180403,상계34동,0.00,112.74,418.45,1292.73,1273.00,1879.52,2023.47,3159.23,...,1664.50,2178.74,2502.02,3333.71,3242.28,3254.22,2525.41,1585.93,1763.33,normal
3,20180404,상계34동,2.65,132.11,510.20,1384.89,1222.98,1837.64,2040.81,3205.01,...,1557.18,2271.59,2485.72,3341.92,2962.14,3234.81,2403.02,1578.96,1628.89,normal
4,20180405,상계34동,0.00,97.59,428.40,1255.86,1197.84,1921.00,1910.03,3005.59,...,1590.45,2134.92,2253.33,3183.49,2716.88,2857.37,2168.00,1360.75,1598.15,normal


In [41]:
age_level = age_level.drop(['HDONG_NM','STD_YMD'], axis = 1)

In [42]:
print(age_level.shape)
age_level.head()

(365, 31)


,MAN_FLOW_POP_CNT_0004,MAN_FLOW_POP_CNT_0509,MAN_FLOW_POP_CNT_1014,MAN_FLOW_POP_CNT_1519,MAN_FLOW_POP_CNT_2024,MAN_FLOW_POP_CNT_2529,MAN_FLOW_POP_CNT_3034,MAN_FLOW_POP_CNT_3539,MAN_FLOW_POP_CNT_4044,MAN_FLOW_POP_CNT_4549,...,WMAN_FLOW_POP_CNT_3034,WMAN_FLOW_POP_CNT_3539,WMAN_FLOW_POP_CNT_4044,WMAN_FLOW_POP_CNT_4549,WMAN_FLOW_POP_CNT_5054,WMAN_FLOW_POP_CNT_5559,WMAN_FLOW_POP_CNT_6064,WMAN_FLOW_POP_CNT_6569,WMAN_FLOW_POP_CNT_70U,level
0,0.00,153.25,713.84,1252.03,1335.48,1974.79,1920.19,2772.93,2819.34,4354.16,...,1637.68,2266.34,2205.38,3273.12,3088.10,3618.61,2513.22,1474.12,1762.03,bad
1,0.00,148.05,452.80,1301.48,1397.54,1800.54,2068.72,3382.82,3220.15,4502.91,...,1698.75,2180.52,2421.76,3264.91,3016.01,3140.79,2345.18,1515.82,1871.65,normal
2,0.00,112.74,418.45,1292.73,1273.00,1879.52,2023.47,3159.23,3193.41,4480.80,...,1664.50,2178.74,2502.02,3333.71,3242.28,3254.22,2525.41,1585.93,1763.33,normal
3,2.65,132.11,510.20,1384.89,1222.98,1837.64,2040.81,3205.01,3298.84,4364.32,...,1557.18,2271.59,2485.72,3341.92,2962.14,3234.81,2403.02,1578.96,1628.89,normal
4,0.00,97.59,428.40,1255.86,1197.84,1921.00,1910.03,3005.59,3074.95,4117.40,...,1590.45,2134.92,2253.33,3183.49,2716.88,2857.37,2168.00,1360.75,1598.15,normal


In [43]:
age_verybad = age_level.groupby('level').get_group('very bad')
age_bad = age_level.groupby('level').get_group('bad')
age_normal = age_level.groupby('level').get_group('normal')

In [44]:
print(age_verybad.shape)
print(age_bad.shape)
print(age_normal.shape)

(21, 31)
(114, 31)
(230, 31)


등분산성 검정

Bartlett / levene / fligner 세 가지 검정 방법
 * H0 : 각 그룹의 분산이 동일하다
 
p-value 가 작으면 H0 기각

p-value 가 크면 H0 기각 X : 등분산성

In [45]:
# var_df 데이터프레임에 각 성연령별 검정 P-value 값 저장
var_df = pd.DataFrame(columns=[],index=['bartlett','levene','fligner'])

for i in range(30):
    
    d1 = age_verybad.iloc[:,i]
    d2 = age_bad.iloc[:,i]
    d3 = age_normal.iloc[:,i]
    
    name = d1.name.split('_')
    title = name[0] + "_" + name[-1]
    
    R1 = sp.stats.bartlett(d1,d2,d3).pvalue
    R2 = sp.stats.levene(d1,d2,d3).pvalue
    R3 = sp.stats.fligner(d1,d2,d3).pvalue
    
    var_df[title] = [R1,R2,R3]

C:\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1852: RuntimeWarning:

divide by zero encountered in log



In [46]:
var_df

,MAN_0004,MAN_0509,MAN_1014,MAN_1519,MAN_2024,MAN_2529,MAN_3034,MAN_3539,MAN_4044,MAN_4549,...,WMAN_2529,WMAN_3034,WMAN_3539,WMAN_4044,WMAN_4549,WMAN_5054,WMAN_5559,WMAN_6064,WMAN_6569,WMAN_70U
bartlett,0.000000,0.040757,0.136406,0.021550,2.177171e-09,0.487877,0.735884,0.609479,0.517314,0.169808,...,0.995384,0.865686,0.512736,0.447910,0.589611,0.403169,0.241557,0.035706,0.030834,0.137031
levene,0.108561,0.162475,0.222851,0.013518,1.514467e-01,0.764173,0.996375,0.650060,0.549574,0.715118,...,0.893019,0.643419,0.358333,0.794842,0.918464,0.971106,0.948117,0.808179,0.520655,0.722494
fligner,0.158475,0.128714,0.051683,0.002838,5.022091e-02,0.725362,0.996385,0.368580,0.538163,0.809336,...,0.799661,0.416240,0.219780,0.799892,0.872555,0.862582,0.996708,0.894305,0.679574,0.303155


In [47]:
var_df > 0.05

,MAN_0004,MAN_0509,MAN_1014,MAN_1519,MAN_2024,MAN_2529,MAN_3034,MAN_3539,MAN_4044,MAN_4549,...,WMAN_2529,WMAN_3034,WMAN_3539,WMAN_4044,WMAN_4549,WMAN_5054,WMAN_5559,WMAN_6064,WMAN_6569,WMAN_70U
bartlett,False,False,True,False,False,True,True,True,True,True,...,True,True,True,True,True,True,True,False,False,True
levene,True,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
fligner,True,True,True,False,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


# 성연령별 ANOVA

ANOVA 분석

H0 : 각 그룹의 평균이 동일하다

H1 : 적어도 하나의 그룹은 평균이 다르다

p-value 가 작으면 H0 기각 : 적어도 하나의 평균은 다르다 (미세먼지 별 그룹이 차이가 의미가 있다)

p-value 가 크면 H0 기각 X : 그룹 별 평균 차이가 없다 (미세먼지에 영향을 받지 않는다)

In [48]:
L = []

for i in range(30):
    
    data = age_level.iloc[:,[i,30]]
    
    name = data.columns[0].split('_')
    title = name[0] + "_" + name[-1]
    
    data.rename(columns={ data.columns[0] : 'value', data.columns[1]:'treatment'},inplace= True)

    print(title)

    model = ols('value ~ C(treatment)',data).fit()
    print(anova_lm(model))
    print('\n')

    if(anova_lm(model)['PR(>F)'][0] < 0.05):
        L.append(title)
        
        # p-value 0.05 이하인 성별_연령대를 L list에 추가
    

L

C:\Anaconda3\lib\site-packages\pandas\core\frame.py:4238: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



MAN_0004
                 df      sum_sq   mean_sq         F    PR(>F)
C(treatment)    2.0    2.716955  1.358477  2.234123  0.108561
Residual      362.0  220.117192  0.608059       NaN       NaN


MAN_0509
                 df         sum_sq       mean_sq          F        PR(>F)
C(treatment)    2.0  139195.986643  69597.993321  28.485343  3.243491e-12
Residual      362.0  884471.469775   2443.291353        NaN           NaN


MAN_1014
                 df        sum_sq        mean_sq         F    PR(>F)
C(treatment)    2.0  4.286517e+05  214325.846628  8.161016  0.000341
Residual      362.0  9.506899e+06   26262.151338       NaN       NaN


MAN_1519
                 df        sum_sq       mean_sq          F        PR(>F)
C(treatment)    2.0  2.330674e+06  1.165337e+06  32.368975  1.166976e-13
Residual      362.0  1.303261e+07  3.600167e+04        NaN           NaN


MAN_2024
                 df        sum_sq        mean_sq          F        PR(>F)
C(treatment)    2.0  1.480271e+06  7401

['MAN_0509',
 'MAN_1014',
 'MAN_1519',
 'MAN_2024',
 'MAN_2529',
 'MAN_3034',
 'MAN_3539',
 'MAN_4044',
 'MAN_4549',
 'MAN_5054',
 'MAN_5559',
 'MAN_6064',
 'MAN_6569',
 'WMAN_0509',
 'WMAN_1014',
 'WMAN_1519',
 'WMAN_2024',
 'WMAN_2529',
 'WMAN_3034',
 'WMAN_3539',
 'WMAN_4044',
 'WMAN_4549',
 'WMAN_5054',
 'WMAN_5559',
 'WMAN_6064',
 'WMAN_6569']

In [49]:
var_df[L] > 0.05

,MAN_0509,MAN_1014,MAN_1519,MAN_2024,MAN_2529,MAN_3034,MAN_3539,MAN_4044,MAN_4549,MAN_5054,...,WMAN_2024,WMAN_2529,WMAN_3034,WMAN_3539,WMAN_4044,WMAN_4549,WMAN_5054,WMAN_5559,WMAN_6064,WMAN_6569
bartlett,False,True,False,False,True,True,True,True,True,True,...,False,True,True,True,True,True,True,True,False,False
levene,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
fligner,True,True,False,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


상계34동은 유의미한 그룹이 너무많은데.....

In [55]:
#세컬럼 전부 TRUE인 애들만 따로 뽑아서 그려봐도 될듯